In [4]:
# Imports
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier

df_players = pd.read_csv('dataset/processed/players_teams_processed.csv')
df_awards = pd.read_csv('dataset/processed/awards_processed.csv')

stats_df_players = df_players[['playerID', 'year', 'tmID']].copy()

stats_df_players['minutes'] = df_players['minutes'] + df_players['PostMinutes']

stats_df_players["Attacking Score"] = np.where(
    df_players['PostMinutes'] == 0,
    df_players['points'] / df_players['minutes'],
    np.where(
        df_players['minutes'] == 0,
        df_players['PostBlocks'] / df_players['PostMinutes'],
        (df_players['points'] / df_players['minutes']) * 0.4 + (df_players['PostPoints'] / df_players['PostMinutes']) * 0.6
    )
)

stats_df_players["Defensive Score"] = np.where(
    df_players['PostMinutes'] == 0,
    df_players['blocks'] / df_players['minutes'],
    np.where(
        df_players['minutes'] == 0,
        df_players['PostBlocks'] / df_players['PostMinutes'],
        (df_players['blocks'] / df_players['minutes']) * 0.4 + (df_players['PostBlocks'] / df_players['PostMinutes']) * 0.6
    )
)

awards_count = df_awards.groupby(['playerID', 'year']).size().reset_index(name='awards_count')
stats_df_players = awards_count.merge(stats_df_players, how="outer", on=["playerID", "year"])

stats_df_players["Awards Count"] = stats_df_players["awards_count"].fillna(0)

#stats_df_players.dropna(axis=0, how='any', inplace=True)
#stats_df_players.isna().sum()

#gajos com 0 minutos
#df_players.loc[df_players['minutes'] <= 0, ['minutes', 'PostMinutes']]

stats_df_players.head()

df_teams = pd.read_csv('dataset/processed/teams_processed.csv')

team_stats = stats_df_players.groupby(['tmID', 'year'])[['Attacking Score', 'Defensive Score', 'Awards Count']].mean().reset_index()

df_teams = pd.merge(df_teams, team_stats, on=['tmID', 'year'], how='left')

df_teams.to_csv('dataset/processed/teams_test.csv', index=False)


df_teams.head()
#df_teams["Defensive Score"] = 
#df_teams["Awards Count"] = 



,year,tmID,franchID,confID,rank,playoff,firstRound,semis,finals,name,...,homeW,homeL,awayW,awayL,confW,confL,min,Attacking Score,Defensive Score,Awards Count
0,9,ATL,ATL,EA,7,N,NaN,NaN,NaN,Atlanta Dream,...,1,16,3,14,2,18,6825,0.317220,0.024429,0.000000
1,10,ATL,ATL,EA,2,Y,L,NaN,NaN,Atlanta Dream,...,12,5,6,11,10,12,6950,0.358332,0.011777,0.076923
2,1,CHA,CHA,EA,8,N,NaN,NaN,NaN,Charlotte Sting,...,5,11,3,13,5,16,6475,0.317736,0.011721,0.000000
3,2,CHA,CHA,EA,4,Y,W,W,L,Charlotte Sting,...,11,5,7,9,15,6,6500,0.249030,0.014848,0.000000
4,3,CHA,CHA,EA,2,Y,L,NaN,NaN,Charlotte Sting,...,11,5,7,9,12,9,6450,0.247860,0.010089,0.000000
